In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

train_data = pd.read_csv("./train.csv")
train_data.head()

test_data = pd.read_csv("./test.csv")


./SpaceshipTitanic.ipynb
./test.csv
./train.csv


In [26]:
train_data.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,PassengerGroup,PassengerNumber,Deck,DeckNumber,DeckSide
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0001,01,B,0,P
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0002,01,F,0,S
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003,01,A,0,S
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0003,02,A,0,S
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0004,01,F,1,S


In [25]:
# Train Data parsing
Seperate_Passenger_ID = train_data["PassengerId"].str.split("_", n=1, expand=True)
train_data["PassengerGroup"]=Seperate_Passenger_ID[0]
train_data["PassengerNumber"]=Seperate_Passenger_ID[1]
train_data.drop(columns=["PassengerId"], inplace=True)

Seperate_Cabin=train_data["Cabin"].str.split("/", n=2, expand=True)
train_data["Deck"]=Seperate_Cabin[0]
train_data["DeckNumber"]=Seperate_Cabin[1]
train_data["DeckSide"]=Seperate_Cabin[2]
train_data.drop(columns=["Cabin"], inplace=True)


KeyError: 'PassengerId'

In [20]:
print(train_data.columns)

Index(['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported',
       'PassengerGroup', 'PassengerNumber', 'Deck', 'DeckNumber', 'DeckSide'],
      dtype='object')


In [22]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Transported"]

features = ['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService','FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name','PassengerGroup', 'PassengerNumber', 'Deck', 'DeckNumber', 'DeckSide']
X = pd.get_dummies(train_data[features])
print(X)
X_test = pd.get_dummies(test_data[features])
print(X_test)
model = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

       Age  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  \
0     39.0          0.0        0.0           0.0     0.0     0.0   
1     24.0        109.0        9.0          25.0   549.0    44.0   
2     58.0         43.0     3576.0           0.0  6715.0    49.0   
3     33.0          0.0     1283.0         371.0  3329.0   193.0   
4     16.0        303.0       70.0         151.0   565.0     2.0   
...    ...          ...        ...           ...     ...     ...   
8688  41.0          0.0     6819.0           0.0  1643.0    74.0   
8689  18.0          0.0        0.0           0.0     0.0     0.0   
8690  26.0          0.0        0.0        1872.0     1.0     0.0   
8691  32.0          0.0     1049.0           0.0   353.0  3235.0   
8692  44.0        126.0     4688.0           0.0     0.0    12.0   

      HomePlanet_Earth  HomePlanet_Europa  HomePlanet_Mars  CryoSleep_False  \
0                False               True            False             True   
1                 True   

KeyError: "['PassengerGroup', 'PassengerNumber', 'Deck', 'DeckNumber', 'DeckSide'] not in index"

In [ ]:
submission = pd.read_csv("./submission.csv")
submission.head()